# Use EcustAutodiff to chanllenge Fashion-MNIST

## Dataset

In [6]:
import gzip
import struct
import random
import numpy as np



Get train labels

In [7]:
with gzip.open('../data/train-labels-idx1-ubyte.gz', 'rb') as f:
    f_labels = f.read()

labels = []
for label in f_labels:
    labels.append(label)
labels = labels[8:]

In [8]:
labels[:20]

[5, 0, 4, 1, 9, 2, 1, 3, 1, 4, 3, 5, 3, 6, 1, 7, 2, 8, 6, 9]

Get train images (it takes few time)

In [10]:
abort = 30000 # 0...60000
with gzip.open('../data/train-images-idx3-ubyte.gz', 'rb') as f:
    f_images = f.read()
data = []
for i in range(16, len(f_images)-784*abort, 784):
    image = []
    for j in range(784):
        image.append(f_images[i+j])
    image = np.array(image)
    label = np.zeros((10))
    label[int(labels[int((i-16)/784)])] = 1
    data.append((image, label))


## Train

Import our autodiff libary

In [11]:
import sys
sys.path.append("..")
import ecust_autodiff as ad

Build a model

In [12]:
x = ad.Variable(name = "x")
W = ad.Variable(name = "W")
b = ad.Variable(name = "b")
y_ = ad.Variable(name = "y_")

y = ad.matmul_op(x,W) + b

loss = ad.softmax_xentropy_op(y, y_)

grad_W, grad_b = ad.gradients(loss, [W, b])

executor = ad.Executor([loss, grad_W, grad_b])


Train the model, (it takes few time)

In [17]:
Ws = np.zeros([784, 10])
bs = np.zeros([10])

lr_W = 1e-8
lr_b = 1e-8

for i in range(10000):
    input_data = random.sample(data, 100)
    input_x = []
    input_y = []
    for j in range(100):
        input_x.append(input_data[j][0])
        input_y.append(input_data[j][1])
    
    input_xs = np.array(input_x)
    input_ys = np.array(input_y)

    loss_val, grad_W_val, grad_b_val = executor.run(feed_dict = {
        x: input_xs,
        W: Ws,
        b: bs,
        y_: input_ys
    })
    
    Ws -= grad_W_val * lr_W
    bs -= np.mean(grad_b_val, 0) * lr_b

    if i%100==0:
        print(loss_val)

[2.30258509]
[0.73759473]
[0.64281801]
[0.60644989]
[0.40923904]
[0.32014206]
[0.40703779]
[0.43270911]
[0.42863058]
[0.28505359]
[0.31245863]
[0.24651658]
[0.36071021]
[0.31274222]
[0.27891548]
[0.34901461]
[0.35514582]
[0.2285766]
[0.37617878]
[0.31967068]
[0.47430461]
[0.33755185]
[0.23954066]
[0.4444178]
[0.3587439]
[0.37348198]
[0.43301166]
[0.28688442]
[0.43431941]
[0.33876803]
[0.24440659]
[0.34306263]
[0.16818926]
[0.27632778]
[0.41019174]
[0.21316098]
[0.66352485]
[0.21775188]
[0.39690728]
[0.26463589]
[0.27008413]
[0.33544858]
[0.25191659]
[0.22618156]
[0.32999534]
[0.27602639]
[0.28249093]
[0.42483457]
[0.37025403]
[0.26054738]
[0.31216168]
[0.26584771]
[0.30963619]
[0.18384146]
[0.23482891]
[0.21765322]
[0.21938448]
[0.275007]
[0.34292783]
[0.3841108]
[0.26028863]
[0.32601418]
[0.2796153]
[0.26294895]
[0.42570862]
[0.45350756]
[0.31472751]
[0.11306225]
[0.22036726]
[0.22758989]
[0.15704856]
[0.2221577]
[0.1813842]
[0.29521664]
[0.23630079]
[0.43597198]
[0.19423148]
[0.19003

## Test

In [19]:
with gzip.open('../data/t10k-labels-idx1-ubyte.gz','rb') as f:
    f_tlabel = f.read()
tlabel = []
for i in f_tlabel:
    tlabel.append(i)
tlabel = tlabel[8:]

with gzip.open('../data/t10k-images-idx3-ubyte.gz','rb') as f:
    f_timage = f.read()
timage = []
for i in range(16,len(f_timage),784):
    temp = []
    for j in range(784):
        temp.append(f_timage[i+j])
    timage.append(temp)

In [20]:
test_xs = []
for i in timage:
    test_xs.append(np.array(i))
test_xs = np.array(test_xs)

test_ys = []
for i in tlabel:
    temp = np.zeros(10)
    temp[i] = 1
    test_ys.append(temp)
test_ys = np.array(test_ys)

test = ad.Executor([y])

In [23]:
ys, = test.run(feed_dict = {
    x: test_xs,
    W: Ws,
    b: bs
})

correct_prediction = np.equal(np.argmax(ys,1), np.argmax(test_ys,1))
accuracy = np.mean(correct_prediction)
print(accuracy)

accuracy : 
0.9172
